In [3]:
import os

In [5]:
os.getcwd()

'C:\\Users\\kaspe'

# data_loader.py
specifies how the data should be fed to the network: LightningDataModule

In short, data preparation has 4 steps:

- Download images
- Image transforms (these are highly subjective).
- Generate training, validation and test dataset splits.
- Wrap each dataset split in a DataLoader


In [ ]:
# https://lightning.ai/docs/pytorch/latest/data/datamodule.html

import lightning.pytorch as pl
from torch.utils.data import random_split, DataLoader

# Note - you must have torchvision installed for this example
from torchvision.datasets import MNIST
from torchvision import transforms


class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "./"):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage: str):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test":
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

        if stage == "predict":
            self.mnist_predict = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=32)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=32)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=32)

    def predict_dataloader(self):
        return DataLoader(self.mnist_predict, batch_size=32)

data_module = MNISTDataModule()

# model.py
specifies the neural network architecture, the loss function and evaluation metrics

In [ ]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import os

class LightningMNISTClassifier(pl.LightningModule):

  def __init__(self):
    super().__init__()

    # mnist images are (1, 28, 28) (channels, width, height) 
    self.layer_1 = torch.nn.Linear(28 * 28, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, 10)

  def forward(self, x):
      batch_size, channels, width, height = x.size()

      # (b, 1, 28, 28) -> (b, 1*28*28)
      x = x.view(batch_size, -1)

      # layer 1 (b, 1*28*28) -> (b, 128)
      x = self.layer_1(x)
      x = torch.relu(x)

      # layer 2 (b, 128) -> (b, 256)
      x = self.layer_2(x)
      x = torch.relu(x)

      # layer 3 (b, 256) -> (b, 10)
      x = self.layer_3(x)

      # probability distribution over labels
      x = torch.log_softmax(x, dim=1)

      return x

  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)

  def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('train_loss', loss)
      return loss


  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('val_loss', loss)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer


model = LightningMNISTClassifier()

# train.py
train models and save them

In [ ]:
---
---
---
trainer.fit(model, data_module)

# Save the model with help from utils.py
utils.save_model(model=model1,
                 target_dir="models",
                 model_name="Scratch.pth")
utils.save_model(model=model2,
                 target_dir="models",
                 model_name="ResNet-18.pth")
utils.save_model(model=model3,
                 target_dir="models",
                 model_name="VGG-16.pth")

# evaluate.py
contains the main loop for evaluating the model

# search_hyperparams.py
hyper parameter search

# synthesize_results.py
An author synthesizes study data by combining the results together to enable comparison and to allow others to draw further conclusions from them

# evaluate.py
contains the main loop for evaluating the model

# utils.py
utility functions for handling hyperparams/logging/storing model

In [ ]:
BATCH_SIZE = 128
NUM_EPOCHS = 20
LEARNING_RATE = 0.005
NUM_WORKERS = 4